In [23]:
import pandas as pd
import json
from tqdm import tqdm
from functions import verifica_dano_ambiental, analisa_sentenca, divide_lista_em_partes
import time

## Obtenção inicial dos dados via juscraper

In [ ]:
# Última atualização: 2025-06-07
tribunais_justica = [
    # "tjac",  # Tribunal de Justiça do Acre - Ainda não implementado
    # "tjal",  # Tribunal de Justiça de Alagoas - Ainda não implementado
    # "tjam",  # Tribunal de Justiça do Amazonas - Ainda não implementado
    # "tjap",  # Tribunal de Justiça do Amapá - Ainda não implementado
    # "tjba",  # Tribunal de Justiça da Bahia - Ainda não implementado
    # "tjce",  # Tribunal de Justiça do Ceará - Ainda não implementado
    # "tjdft", # Tribunal de Justiça do Distrito Federal e dos Territórios - Ainda não implementado
    # "tjes",  # Tribunal de Justiça do Espírito Santo - Ainda não implementado
    # "tjgo",  # Tribunal de Justiça de Goiás - Ainda não implementado
    # "tjma",  # Tribunal de Justiça do Maranhão - Ainda não implementado
    # "tjmg",  # Tribunal de Justiça de Minas Gerais - Ainda não implementado
    # "tjms",  # Tribunal de Justiça de Mato Grosso do Sul - Ainda não implementado
    # "tjmt",  # Tribunal de Justiça de Mato Grosso - Ainda não implementado
    # "tjpa",  # Tribunal de Justiça do Pará - Ainda não implementado
    # "tjpb",  # Tribunal de Justiça da Paraíba - Ainda não implementado
    # "tjpe",  # Tribunal de Justiça de Pernambuco - Ainda não implementado
    # "tjpi",  # Tribunal de Justiça do Piauí - Ainda não implementado
    # "tjrj",  # Tribunal de Justiça do Rio de Janeiro - Ainda não implementado
    # "tjrn",  # Tribunal de Justiça do Rio Grande do Norte - Ainda não implementado
    # "tjro",  # Tribunal de Justiça de Rondônia - Ainda não implementado
    # "tjrr",  # Tribunal de Justiça de Roraima - Ainda não implementado
    "tjrs",  # Tribunal de Justiça do Rio Grande do Sul - Implementado parcialmente
    # "tjsc",  # Tribunal de Justiça de Santa Catarina - Ainda não implementado
    # "tjse",  # Tribunal de Justiça de Sergipe - Ainda não implementado
    # "tjto",  # Tribunal de Justiça do Tocantins - Ainda não implementado
    "tjsp",  # Tribunal de Justiça de São Paulo - Implementado
]

In [11]:
# Extração inicial até a página 5000 do juscraper
import juscraper as jus
tjsp = jus.scraper('tjsp')
dados_cjpg = tjsp.cjpg(pesquisa='dano ambiental', paginas=range(1,5000))

dados_cjpg

Processando documentos: 100%|██████████| 4110/4110 [03:55<00:00, 17.48it/s]


,cd_processo,id_processo,classe,assunto,magistrado,comarca,foro,vara,data_disponibilizacao,decisao
0,C90004GBC0000,1000882-09.2025.8.26.0441,Procedimento Comum Cível,Indenização por Dano Moral,GUILHERME PINHO RIBEIRO,Peruíbe,Foro de Peruíbe,1ª Vara,26/05/2025,SENTENÇA\n\n\n\nProcesso Digital nº:\t1000882-...
1,1H000TLSP0000,1094870-21.2024.8.26.0053,Procedimento Comum Cível,Dano Ambiental,José Roberto Leme Alves de Oliveira,SÃO PAULO,Foro Central - Fazenda Pública/Acidentes,6ª Vara de Fazenda Pública,26/05/2025,SENTENÇA\n\n\n\nProcesso Digital nº:\t1094870-...
2,23Z0600F90000,0000549-72.2006.8.26.0075,Ação Civil Pública,Dano Ambiental,Jade Marguti Cidade,Bertioga,Foro de Bertioga,1ª Vara,26/05/2025,SENTENÇA\n\n\n\nProcesso Digital nº:\t0000549-...
3,1W001CV9X0000,1025287-35.2023.8.26.0068,Procedimento do Juizado Especial Cível,Indenização por Dano Moral,Telma Berkelmans dos Santos,Barueri,Foro de Barueri,1ª Vara do Juizado Especial Cível e Criminal,26/05/2025,SENTENÇA\n\n\n\nProcesso Digital nº:\t1025287-...
4,8Y0004C680000,0003078-69.2024.8.26.0322,Ação Penal - Procedimento Sumaríssimo,Da Poluição,RENATA TEODORO ANDREOLI,Lins,Foro de Lins,2ª Vara Criminal,26/05/2025,SENTENÇA\n\n\n\nProcesso Digital nº:\t0003078-...
...,...,...,...,...,...,...,...,...,...,...
40985,2SZX32OQX0000,0548041-35.2000.8.26.0100,Procedimento Comum Cível,Assunto não informado.,Cláudia Grieco Tabosa Pessoa,SÃO PAULO,Foro Central Cível,14ª Vara Cível,29/08/2003,"VISTOS. \tDARIA DE JESUS SOUZA, qualific..."
40986,2SZX4K5XE0000,0043074-96.2003.8.26.0100,Consignação em Pagamento,Assunto não informado.,Márcio Teixeira Laranjo,SÃO PAULO,Foro Central Cível,24ª Vara Cível,31/07/2003,Ação: Consignatória (em geral) Requerente: P...
40987,3AY0101120000,0001334-75.2001.8.26.0118,Ação Penal - Procedimento Sumaríssimo,Crimes contra a Fauna,Carmen Silvia de Paula Camargo,Cananéia,Foro de Cananéia,Vara Única,20/05/2003,PROCESSO nº 109/2001.\n\n\n\n\n\n\n\n\n\n\t\t...
40988,2SZX40E3R0000,0120551-35.2002.8.26.0100,NaN,Assunto não informado.,CELINA DIETRICH E TRIGUEIROS TEIXEIRA PINTO,SÃO PAULO,Foro Central Cível,2ª Vara Cível,12/11/2002,Contr. 1894 - Fls. 95/99 - Vistos. KOLETA AMBI...


In [12]:
dados_cjpg.shape

(40990, 10)

In [ ]:
dados_cjpg = dados_cjpg.loc[dados_cjpg["assunto"] == "Dano Ambiental", :]
# dados_cjpg.to_excel('docs/juscraper/dados_juscraper.xlsx', index=False)

## Teste inicial da biblioteca

In [ ]:
dados_cjpg = pd.read_excel('docs/juscraper/dados_juscraper.xlsx')

In [ ]:
sentenca_lista = dados_cjpg.loc[dados_cjpg['assunto'].str.contains('dano ambiental', case=False), :].iloc[0, 9].split('\n')
texto = "".join(sentenca_lista).strip()

In [12]:
texto = texto.replace('\t', ' ')
texto

'SENTENÇAProcesso Digital nº: 1002294-33.2023.8.26.0024Classe - Assunto Ação Civil Pública - Dano Ambiental Requerente: Fazenda Pública do Estado de São PauloRequerido: Marinalva de Fatima dos SantosJuiz(a) de Direito: Dr(a). GUILHERME FACCHINI BOCCHI AZEVEDOVistos.Trata-se de Ação Civil Pública Ambiental ajuizada pela FAZENDA PÚBLICA DO ESTADO DE SÃO PAULO em face de MARINALVA DE FÁTIMA DOS SANTOS, objetivando a condenação da requerida à recuperação de área de preservação permanente (APP) degradada. Segundo a inicial, no dia 06 de março de 2004, durante fiscalização ambiental realizada no Bairro Porto Independência, município de Castilho/SP, foi constatada construção, reforma e ampliação em alvenaria no imóvel denominado "Rancho do Hugo", suprimindo vegetação natural em área de 0,0192 hectares, considerada de preservação permanente por estar a menos de 100 metros do reservatório da Usina Hidrelétrica Engenheiro Sergio Motta, o que motivou a lavratura do Auto de Infração Ambiental nº 1

In [15]:
analise = analisa_sentenca(texto)
analise

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='{\n"numero_processo": "1002294-33.2023.8.26.0024",\n"georreferencia": "NULL",\n"uf": "SP",\n"municipio": "Castilho",\n"responsavel": "Marinalva de Fatima dos Santos",\n"categoria_responsavel": "Pessoa Física",\n"tipo_impacto": "Desmatamento de APP",\n"descricao_impacto": "Construção, reforma e ampliação em alvenaria, suprimindo vegetação natural em área de preservação permanente.",\n"data_impacto": "06/03/04",\n"area_afetada": "192",\n"unidade_area": "m2",\n"houve_compensacao": true,\n"categoria_compensacao": "Obrigações de Fazer (com custo)",\n"tipo_multa": 2,\n"valor_multa": "NULL",\n"valor_multa_diaria": "500.00"\n}')], role='model'), citation_metadata=None, finish_message=None, token_count=None, finish_reason=<FinishReason.STOP: 'STOP'>, avg_l

In [16]:
json.loads(analise.text)  

{'numero_processo': '1002294-33.2023.8.26.0024',
 'georreferencia': 'NULL',
 'uf': 'SP',
 'municipio': 'Castilho',
 'responsavel': 'Marinalva de Fatima dos Santos',
 'categoria_responsavel': 'Pessoa Física',
 'tipo_impacto': 'Desmatamento de APP',
 'descricao_impacto': 'Construção, reforma e ampliação em alvenaria, suprimindo vegetação natural em área de preservação permanente.',
 'data_impacto': '06/03/04',
 'area_afetada': '192',
 'unidade_area': 'm2',
 'houve_compensacao': True,
 'categoria_compensacao': 'Obrigações de Fazer (com custo)',
 'tipo_multa': 2,
 'valor_multa': 'NULL',
 'valor_multa_diaria': '500.00'}

## Repetindo o processo de classificação usado no JusBrasil 

In [ ]:
processos_lista = pd.read_excel('docs/juscraper/dados_juscraper.xlsx')
processos_lista.head(5)

,cd_processo,id_processo,classe,assunto,magistrado,comarca,foro,vara,data_disponibilizacao,decisao
0,1H000TLSP0000,1094870-21.2024.8.26.0053,Procedimento Comum Cível,Dano Ambiental,José Roberto Leme Alves de Oliveira,SÃO PAULO,Foro Central - Fazenda Pública/Acidentes,6ª Vara de Fazenda Pública,26/05/2025,SENTENÇA\n\n\n\nProcesso Digital nº:\t1094870-...
1,23Z0600F90000,0000549-72.2006.8.26.0075,Ação Civil Pública,Dano Ambiental,Jade Marguti Cidade,Bertioga,Foro de Bertioga,1ª Vara,26/05/2025,SENTENÇA\n\n\n\nProcesso Digital nº:\t0000549-...
2,1Z000LZSG0000,0001330-42.2025.8.26.0071,Procedimento Comum Cível,Dano Ambiental,Elaine Cristina Storino Leoni,Bauru,Foro de Bauru,2ª Vara da Fazenda Pública,26/05/2025,SENTENÇA\n\n\n\nProcesso nº:\t0001330-42.2025....
3,HU0000WJZ0000,1000125-09.2016.8.26.0642,Ação Civil Pública,Dano Ambiental,Leonardo Grecco,Ubatuba,Foro de Ubatuba,2ª Vara,23/05/2025,SENTENÇA\n\n\n\nProcesso Digital nº:\t1000125-...
4,570002CJ80000,0000073-56.2024.8.26.0187,Liquidação de Sentença pelo Procedimento Comum,Dano Ambiental,Roberta de Oliveira Ferreira Lima,Fartura,Foro de Fartura,Vara Única,23/05/2025,"CONCLUSÃO\n\nAos 23 de maio de 2.025, faço est..."


In [ ]:
processos_lista.shape

(2470, 10)

In [ ]:
lista_processos = list(processos_lista['id_processo'].unique())

partes = divide_lista_em_partes(lista_processos, 10)
print("Distribuição de processos entre as partes:")
for i, parte in enumerate(partes):
    print(f"Parte {i + 1}: {len(parte)} processos")
print("Total de processos:", len(lista_processos))

Distribuição de processos entre as partes:
Parte 1: 220 processos
Parte 2: 220 processos
Parte 3: 220 processos
Parte 4: 220 processos
Parte 5: 220 processos
Parte 6: 220 processos
Parte 7: 220 processos
Parte 8: 220 processos
Parte 9: 220 processos
Parte 10: 221 processos
Total de processos: 2201


In [ ]:
# Fazendo uma verificação adicional para garantir que o texto trate de dano ambiental
respostas_final = pd.DataFrame()

for i, parte in enumerate(partes):
    print(f"Parte {i + 1}: {len(parte)} processos")
    
    respostas = []

    for id in tqdm(parte):
        linha = processos_lista.loc[processos_lista['id_processo'] == id]
        linha = linha.iloc[0, 9] # Capturando a decisão
        if isinstance(linha, str):
            texto = linha.strip().replace('\t', ' ')
        elif isinstance(linha, (list, tuple)):
            texto = " ".join(str(x) for x in linha).strip().replace('\t', ' ')
        else:
            texto = str(linha).strip().replace('\t', ' ')

        resposta_prompt = verifica_dano_ambiental(texto)
        resposta_prompt = json.loads(resposta_prompt.text)
        resposta_prompt["processoID"] = id
        resposta_prompt["decisao"] = texto
        respostas.append(resposta_prompt)
    
    respostas_novas = pd.DataFrame(respostas)
    respostas_final = pd.concat([respostas_final, respostas_novas], ignore_index=True)
    # respostas_final.to_excel("docs/juscraper/respostas_classificacao_juscraper.xlsx", index=False)
    time.sleep(5)

Parte 1: 220 processos


  0%|          | 0/220 [00:00<?, ?it/s]

100%|██████████| 220/220 [1:07:56<00:00, 18.53s/it]


Parte 2: 220 processos


100%|██████████| 220/220 [1:09:35<00:00, 18.98s/it]


...

100%|██████████| 220/220 [14:42<00:00,  4.01s/it]


Parte 9: 220 processos


100%|██████████| 220/220 [28:09<00:00,  7.68s/it]


Parte 10: 221 processos


100%|██████████| 221/221 [38:10<00:00, 10.36s/it]


In [2]:
respostas_final = pd.read_excel("docs/juscraper/respostas_classificacao_juscraper.xlsx")
respostas_final.shape

(2201, 4)

In [3]:
respostas_final.loc[respostas_final["isDanoAmbiental"] == True, :]

,isDanoAmbiental,justificativa,processoID,decisao
0,True,Houve supressão irregular de 11 árvores em áre...,1094870-21.2024.8.26.0053,SENTENÇA\n\n\n\nProcesso Digital nº: 1094870-2...
1,True,Aterro de vegetação de preservação permanente ...,0000549-72.2006.8.26.0075,SENTENÇA\n\n\n\nProcesso Digital nº: 0000549-7...
3,True,"Houve desmatamento, construções irregulares, i...",1000125-09.2016.8.26.0642,SENTENÇA\n\n\n\nProcesso Digital nº: 1000125-0...
4,True,A sentença transitada em julgado determinou a ...,0000073-56.2024.8.26.0187,"CONCLUSÃO\n\nAos 23 de maio de 2.025, faço est..."
5,True,"O texto menciona ""Cumprimento de sentença - Da...",0001617-97.2023.8.26.0063,DECISÃO\n\n\n\nProcesso Digital nº: 0001617-97...
...,...,...,...,...
2170,True,A morte de milhares de peixes devido à redução...,1000903-64.2019.8.26.0515,SENTENÇA\n\n\n\nProcesso Digital nº: 1000903-6...
2171,True,A morte de milhares de peixes de diferentes es...,1001784-41.2019.8.26.0515,SENTENÇA\n\n\n\nProcesso Digital nº: 1001784-4...
2178,True,"Construção irregular em área de proteção, impa...",1001689-62.2017.8.26.0650,SENTENÇA\n\n\n\nProcesso nº: 1001689-62.2017.8...
2179,True,O texto descreve loteamento irregular em área ...,0002247-90.2012.8.26.0338,SENTENÇA\n\n\n\nProcesso Físico nº: 0002247-90...


In [4]:
respostas_final.loc[respostas_final["justificativa"] == "Erro na classificação automática", :].shape

(29, 4)

In [ ]:
# Tentando novamente para corrigir os casos de erro na classificação (chaves inválidas temporariamente)
respostas_final = pd.read_excel("docs/juscraper/respostas_classificacao_juscraper.xlsx")
erros_requisicao = respostas_final.loc[respostas_final["justificativa"] == "Erro na classificação automática", :]
lista_ids = erros_requisicao['processoID'].tolist()
respostas = []

# Segunda rodada com processos em que houve erros de requisição
for id in tqdm(lista_ids):
    linha = erros_requisicao.loc[erros_requisicao['processoID'] == id]
    linha = linha.iloc[0, 3]
    if isinstance(linha, str):
        texto = linha.strip().replace('\t', ' ')
    elif isinstance(linha, (list, tuple)):
        texto = " ".join(str(x) for x in linha).strip().replace('\t', ' ')
    else:
        texto = str(linha).strip().replace('\t', ' ')

    resposta_prompt = verifica_dano_ambiental(texto)
    resposta_prompt = json.loads(resposta_prompt.text)
    resposta_prompt["processoID"] = id
    resposta_prompt["decisao"] = texto
    respostas.append(resposta_prompt)

respostas_final_2 = pd.DataFrame(respostas)

100%|██████████| 29/29 [10:56<00:00, 22.63s/it]


In [6]:
print(respostas_final_2.loc[respostas_final_2["justificativa"] == "Erro na classificação automática", :].count())
print("*"*50)
print(respostas_final_2.shape)

isDanoAmbiental    0
justificativa      0
processoID         0
decisao            0
dtype: int64
**************************************************
(29, 4)


In [ ]:
# Dropando erros de requisição para incluir a segunda rodada
respostas_final = respostas_final.loc[respostas_final["justificativa"] != "Erro na classificação automática", :]
print(respostas_final.shape)

(2172, 4)


In [ ]:
# Substituindo erros de requisição com a segunda rodada classificada
respostas_final = pd.concat([respostas_final, respostas_final_2], ignore_index=True)

In [ ]:
# Visualizando quantos documentos da coluna de decisão não contém o termo "SENTENÇA"
print(respostas_final.shape)
print(respostas_final[respostas_final['decisao'].str.contains(r'SENTENÇA', na=False)].shape)

(2201, 4)
(2126, 4)


In [ ]:
# Filtrando apenas sentenças - sim, faria mais sentido isto vir antes...
respostas_final = respostas_final[respostas_final['decisao'].str.contains(r'SENTENÇA', na=False)]
# respostas_final.to_excel("docs/juscraper/respostas_classificacao_juscraper.xlsx", index=False)

## Repetindo o processo de captura dos dados usado no JusBrasil após a classificação

In [24]:
respostas_df = pd.read_excel('docs/juscraper/respostas_classificacao_juscraper.xlsx')
danos_ambientais = respostas_df.loc[respostas_df["isDanoAmbiental"] == True]

lista_processos = list(danos_ambientais['processoID'].unique())

partes = divide_lista_em_partes(lista_processos, 4)
print("Distribuição de processos entre as partes:")
for i, parte in enumerate(partes):
    print(f"Parte {i + 1}: {len(parte)} processos")
print("Total de processos:", len(lista_processos))

Distribuição de processos entre as partes:
Parte 1: 361 processos
Parte 2: 361 processos
Parte 3: 361 processos
Parte 4: 363 processos
Total de processos: 1446


In [ ]:
# Analisando os processos com dano ambiental
juscraper_final = pd.DataFrame()

for i, parte in enumerate(partes):
    print(f"Parte {i + 1}: {len(parte)} processos")
    
    respostas_danos_ambientais = []

    for id in tqdm(parte):
        linha = danos_ambientais.loc[danos_ambientais['processoID'] == id]
        linha = linha.iloc[0, 3]

        if isinstance(linha, str):
            texto = linha.strip().replace('\t', ' ')
        elif isinstance(linha, (list, tuple)):
            texto = " ".join(str(x) for x in linha).strip().replace('\t', ' ')
        else:
            texto = str(linha).strip().replace('\t', ' ')

        resposta_prompt = analisa_sentenca(texto)
        resposta_prompt = json.loads(resposta_prompt.text)
        resposta_prompt["processoID"] = id
        resposta_prompt["referencia"] = texto
        respostas_danos_ambientais.append(resposta_prompt)

    respostas_novas = pd.DataFrame(respostas_danos_ambientais)
    juscraper_final = pd.concat([juscraper_final, respostas_novas], ignore_index=True)
    # juscraper_final.to_excel("docs/juscraper/respostas_danos_ambientais_juscraper.xlsx", index=False)
    time.sleep(5)

juscraper_final

Parte 1: 361 processos


100%|██████████| 361/361 [47:16<00:00,  7.86s/it]    


Parte 2: 361 processos


100%|██████████| 361/361 [30:50<00:00,  5.13s/it]


Parte 3: 361 processos


100%|██████████| 361/361 [40:11<00:00,  6.68s/it]


Parte 4: 363 processos


100%|██████████| 363/363 [1:08:29<00:00, 11.32s/it]


,numero_processo,georreferencia,uf,municipio,responsavel,categoria_responsavel,tipo_impacto,descricao_impacto,data_impacto,area_afetada,unidade_area,houve_compensacao,categoria_compensacao,tipo_multa,valor_multa,valor_multa_diaria,processoID,referencia
0,1094870-21.2024.8.26.0053,NULL,SP,São Paulo,Condominio Edificio Residencial Espanha,Pessoa Jurídica,Desmatamento de APP,Supressão irregular de 11 árvores em área de m...,NULL,NULL,NULL,True,Obrigações de Fazer (com custo),2,20000.00,500.00,1094870-21.2024.8.26.0053,SENTENÇA\n\n\n\nProcesso Digital nº: 1094870-2...
1,0000549-72.2006.8.26.0075,NULL,SP,Bertioga,Itamar Ferreira Damiao,Pessoa Física,Aterramento de vegetação de preservação perman...,"Aterramento, sem licença, de 1080 m² de vegeta...",NULL,1080,m2,True,Obrigações de Fazer (com custo),0,NULL,NULL,0000549-72.2006.8.26.0075,SENTENÇA\n\n\n\nProcesso Digital nº: 0000549-7...
2,1000125-09.2016.8.26.0642,NULL,SP,Ubatuba,José Damian Canbriani,Pessoa Física,Desmatamento de APP,Desmatamento e construção sem autorização em á...,NULL,NULL,NULL,True,Obrigações de Fazer (com custo),0,NULL,NULL,1000125-09.2016.8.26.0642,SENTENÇA\n\n\n\nProcesso Digital nº: 1000125-0...
3,0000073-56.2024.8.26.0187,NULL,SP,Fartura,Luiz Carlos Milani e Moacir Domingues Filho,Pessoa Física,Dano Ambiental,Restauração da vegetação nativa nos lotes 03 e...,NULL,NULL,NULL,True,Obrigações de Fazer (com custo),2,29702.40,1000.00,0000073-56.2024.8.26.0187,"CONCLUSÃO\n\nAos 23 de maio de 2.025, faço est..."
4,1001500-33.2019.8.26.0515,NULL,MS,Três Lagoas,CESP - COMPANHIA ENERGÉTICA DE SÃO PAULO,Pessoa Jurídica,Morte de peixes por falta de energia elétrica,Morte de milhares de peixes de espécies difere...,10/11/2009,NULL,NULL,False,NULL,0,NULL,NULL,1001500-33.2019.8.26.0515,SENTENÇA\n\n\n\nProcesso Digital nº: 1001500-3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,1001763-65.2019.8.26.0515,NULL,SP,Três Lagoas,CESP - COMPANHIA ENERGÉTICA DE SÃO PAULO,Pessoa Jurídica,Morte de peixes por blecaute em usina hidrelét...,Blecaute na Usina Hidrelétrica Eng. Sousa Dias...,10/11/2009,NULL,NULL,False,NULL,0,NULL,NULL,1001763-65.2019.8.26.0515,SENTENÇA\n\n\n\nProcesso Digital nº: 1001763-6...
1442,0004069-31.2019.8.26.0642,NULL,SP,Ubatuba,Angela de Fatima Lafuente,Pessoa Física,Dano Ambiental,NULL,NULL,NULL,NULL,False,Custas Judiciais e Acordos,0,NULL,NULL,0004069-31.2019.8.26.0642,SENTENÇA\n\nProcesso nº: 0004069-31.2019.8.26....
1443,1001742-89.2019.8.26.0515,NULL,MS,Três Lagoas,CESP - COMPANHIA ENERGÉTICA DE SÃO PAULO,Pessoa Jurídica,Morte de peixes por falta de energia em usina ...,Blecaute na Usina Hidrelétrica Eng. Sousa Dias...,10/11/2009,NULL,NULL,False,NULL,0,NULL,NULL,1001742-89.2019.8.26.0515,SENTENÇA\n\n\n\nProcesso Digital nº: 1001742-8...
1444,0025168-88.2013.8.26.0053,NULL,SP,São Paulo,Chimen Amazonas Bar e Lanches Ltda - EPP,Pessoa Jurídica,Poluição Sonora,Emissão ilegal de ruídos em horários impróprio...,NULL,NULL,NULL,False,Custas Judiciais e Acordos,0,1711500.00,NULL,0025168-88.2013.8.26.0053,SENTENÇA\n\n\n\nProcesso Físico nº: 0025168-88...


In [27]:
juscraper_final.loc[juscraper_final['valor_multa'] != "NULL"]

,numero_processo,georreferencia,uf,municipio,responsavel,categoria_responsavel,tipo_impacto,descricao_impacto,data_impacto,area_afetada,unidade_area,houve_compensacao,categoria_compensacao,tipo_multa,valor_multa,valor_multa_diaria,processoID,referencia
0,1094870-21.2024.8.26.0053,NULL,SP,São Paulo,Condominio Edificio Residencial Espanha,Pessoa Jurídica,Desmatamento de APP,Supressão irregular de 11 árvores em área de m...,NULL,NULL,NULL,True,Obrigações de Fazer (com custo),2,20000.00,500.00,1094870-21.2024.8.26.0053,SENTENÇA\n\n\n\nProcesso Digital nº: 1094870-2...
3,0000073-56.2024.8.26.0187,NULL,SP,Fartura,Luiz Carlos Milani e Moacir Domingues Filho,Pessoa Física,Dano Ambiental,Restauração da vegetação nativa nos lotes 03 e...,NULL,NULL,NULL,True,Obrigações de Fazer (com custo),2,29702.40,1000.00,0000073-56.2024.8.26.0187,"CONCLUSÃO\n\nAos 23 de maio de 2.025, faço est..."
11,0001842-93.2013.8.26.0346,NULL,SP,Martinópolis,PREFEITURA MUNICIPAL DE MARTINOPOLIS,Pessoa Jurídica,Poluição Sonora,Nível de ruído gerado na Escola Municipal caus...,NULL,NULL,NULL,True,Obrigações de Fazer (com custo),0,5000.00,NULL,0001842-93.2013.8.26.0346,SENTENÇA\n\n\n\nProcesso Físico nº: 0001842-93...
21,1000920-92.2017.8.26.0411,NULL,SP,Pacaembu,PREFEITURA MUNICIPAL DE PACAEMBU,Pessoa Jurídica,Disposição Irregular de Resíduos Sólidos,"Disposição inadequada de resíduos sólidos, cau...",NULL,NULL,NULL,True,Obrigações de Fazer (com custo),0,10000.00,NULL,1000920-92.2017.8.26.0411,SENTENÇA\n\n\n\nProcesso Digital nº: 1000920-9...
23,1003036-20.2019.8.26.0664,NULL,SP,Fernandópolis,Ed Carlos David Bento,Pessoa Física,Maus-tratos a animais,"Participação em rinha de galo, resultando em m...",NULL,NULL,NULL,False,Multas Administrativas,0,3000.00,NULL,1003036-20.2019.8.26.0664,SENTENÇA\n\n\n\nProcesso nº: 1003036-20.2019.8...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1425,1000367-95.2019.8.26.0695,NULL,SP,Nazare Paulista,Carvalheira Industria de Papeis Ltda,Pessoa Jurídica,Poluição Sonora,Emissão de ruídos acima dos limites estabeleci...,23/07/2018,NULL,NULL,True,Obrigações de Fazer (com custo),2,30000.00,1000.00,1000367-95.2019.8.26.0695,SENTENÇA\n\n\n\nProcesso Digital nº: 1000367-9...
1431,1032528-13.2013.8.26.0100,NULL,SP,São Paulo,Liquigás Distribuidora S.A.,Pessoa Jurídica,Contaminação do solo,Contaminação do solo por ausência de troca de ...,15/06/95,NULL,NULL,False,Custas Judiciais e Acordos,0,1735250.00,NULL,1032528-13.2013.8.26.0100,"TERMO DE CONCLUSÃO\n\n\n\nEm , faço estes auto..."
1433,1004973-07.2019.8.26.0554,NULL,SP,Santo André,SERVICO MUNICIPAL DE SANEAMENTO AMBIENTAL DE S...,Pessoa Jurídica,Poluição Hídrica,Esgoto invadiu residência devido ao refluxo da...,19/05/NULL,NULL,NULL,False,Custas Judiciais e Acordos,0,8000.00,NULL,1004973-07.2019.8.26.0554,SENTENÇA\n\n\n\nProcesso Digital nº: 1004973-0...
1444,0025168-88.2013.8.26.0053,NULL,SP,São Paulo,Chimen Amazonas Bar e Lanches Ltda - EPP,Pessoa Jurídica,Poluição Sonora,Emissão ilegal de ruídos em horários impróprio...,NULL,NULL,NULL,False,Custas Judiciais e Acordos,0,1711500.00,NULL,0025168-88.2013.8.26.0053,SENTENÇA\n\n\n\nProcesso Físico nº: 0025168-88...
